# Model 

In [1]:
import pandas as pd # type: ignore
import numpy as np   # type: ignore
from sklearn.pipeline import Pipeline    # type: ignore
from sklearn.preprocessing import StandardScaler, OrdinalEncoder  # type: ignore
from sklearn.compose import ColumnTransformer  # type: ignore
from lightgbm import LGBMClassifier # type: ignore
import joblib

# Load data
train_sample = pd.read_csv("train.csv")
train_orgin_extra = pd.read_csv("ObesityDataSet.csv")
train = pd.concat([train_sample, train_orgin_extra], ignore_index=True)
train.drop('id', axis=1, inplace=True)

# Remove duplicated data
train = train.drop_duplicates()

# Define categorical columns
categorical_columns = train.select_dtypes(include='object').columns
if 'NObeyesdad' in categorical_columns:
    categorical_columns = categorical_columns.drop(['NObeyesdad'])

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), categorical_columns),
        ('scaling', StandardScaler(), ['Age', 'Weight'])
    ], remainder='passthrough'
)

# Define pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        num_class=7,
        verbosity=-1,
        boosting_type='gbdt',
        random_state=42,
        learning_rate=0.03276219058578542,
        n_estimators=500,
        lambda_l1=0.009879324515507773,
        lambda_l2=0.04509276523818003,
        max_depth=10,
        colsample_bytree=0.451686663982718,
        subsample=0.9636469087931024,
        min_child_samples=28,
        n_jobs=-1
    ))
])

# Prepare data
X = train.drop(['NObeyesdad'], axis=1)
y = train['NObeyesdad']

# Train model
pipeline.fit(X, y)



FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
# Save the pipeline to a file
joblib.dump(pipeline, 'model_pipeline.joblib')

# Load the pipeline from the file
pipeline_loaded = joblib.load('model_pipeline.joblib')

test_sample = pd.read_csv("test.csv")

# Make predictions on the test set
y_pred = pipeline_loaded.predict(test_sample)
y_pred

In [48]:
for cat in  categorical_columns :
    print(f"{cat}  "  ,train[cat].unique() )

Gender   ['Male' 'Female']
family_history_with_overweight   ['yes' 'no']
FAVC   ['yes' 'no']
CAEC   ['Sometimes' 'Frequently' 'no' 'Always']
SMOKE   ['no' 'yes']
SCC   ['no' 'yes']
CALC   ['Sometimes' 'no' 'Frequently' 'Always']
MTRANS   ['Public_Transportation' 'Automobile' 'Walking' 'Motorbike' 'Bike']


In [25]:
import joblib

# Load the saved pipeline
pipeline_loaded = joblib.load('model_pipeline.joblib')

# Print the type of the loaded object
print(type(pipeline_loaded))

<class 'sklearn.pipeline.Pipeline'>


In [59]:
# import pandas as pd
# import numpy as np
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder
# from sklearn.compose import ColumnTransformer
# from lightgbm import LGBMClassifier

# # Load data
# train_sample = pd.read_csv("train.csv")
# train_orgin_extra = pd.read_csv("ObesityDataSet.csv")
# train = pd.concat([train_sample, train_orgin_extra], ignore_index=True)
# train.drop('id', axis=1, inplace=True)

# # Remove duplicated data
# train = train.drop_duplicates()

# # Define categorical columns
# categorical_columns = train.select_dtypes(include='object').columns
# if 'NObeyesdad' in categorical_columns:
#     categorical_columns = categorical_columns.drop(['NObeyesdad'])

# # Define preprocessing steps
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('ordinal', OrdinalEncoder(), categorical_columns),
#         ('scaling', StandardScaler(), ['Age', 'Weight'])
#     ], remainder='passthrough'
# )

# # Define pipeline
# pipeline = Pipeline([
#     ('preprocessor', preprocessor),
#     ('classifier', LGBMClassifier(
#         objective='multiclass',
#         metric='multi_logloss',
#         num_class=7,
#         verbosity=-1,
#         boosting_type='gbdt',
#         random_state=42,
#         learning_rate=0.03276219058578542,
#         n_estimators=500,
#         lambda_l1=0.009879324515507773,
#         lambda_l2=0.04509276523818003,
#         max_depth=10,
#         colsample_bytree=0.451686663982718,
#         subsample=0.9636469087931024,
#         min_child_samples=28,
#         n_jobs=-1
#     ))
# ])

# # Prepare data
# X = train.drop(['NObeyesdad'], axis=1)
# y = train['NObeyesdad']

# # Train model
# pipeline.fit(X, y)








############################################################################################



import dash
from dash import Input, Output, State, html, dcc, dash_table, MATCH, ALL, ctx
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc
from dash_iconify import DashIconify
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, time, timedelta
import time as time_pck
import os
import dash_daq as daq
import joblib  # Import joblib to load the trained model

app = dash.Dash(external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])




# HOME_PATH = os.getcwd()
# DATA_PATH = os.path.join(HOME_PATH, 'data')
# MODELS_PATH = "D:/Projects ITI/DV/best_lgb_model.pkl"

# Now you can use this file_path variable wherever you need it
pipeline_loaded = joblib.load('model_pipeline.joblib')




# Gender   ['Male' 'Female']
# family_history_with_overweight   ['yes' 'no']
# FAVC   ['yes' 'no']
# CAEC   ['Sometimes' 'Frequently' 'no' 'Always']
# SMOKE   ['no' 'yes']
# SCC   ['no' 'yes']
# CALC   ['Sometimes' 'no' 'Frequently' 'Always']
# MTRANS   ['Public_Transportation' 'Automobile' 'Walking' 'Motorbike' 'Bike']






# Define custom styles
custom_button_style = {
    'background-color': '#f0f0f0',
    'border-radius': '10px',
    'padding': '10px',
    'margin-bottom': '5px',
    'text-decoration': 'none',
    'display': 'inline-block',
}

Div_style = {
    'padding': '20px',
    'box-shadow': '0 4px 8px 0 rgba(0,0,0,0.2)',
    'marginTop': '10px',
    'backgroundColor': 'rgba(217, 218, 219, 0.6)',
    'color': 'black',
    'borderRadius': '25px',
    'display': 'flex',
    'justifyContent': 'space-between',  # Align divs horizontally
}

# App layout
app.layout = html.Div(children=[
                 ## first Div with drop down
                 html.Div(
                    [


                        html.Div(
                              [dbc.Label("Gender"),
                              dcc.Dropdown(
                                id='gender-dropdown',
                                options=[
                                    
                                    {'label': 'Female', 'value': 'Female'},
                                    {'label': 'Male', 'value': 'Male'}

                                ],
                                value='Female'
                            )]
                            , className='three columns'),


                        html.Div(
                            [dbc.Label("Age"),
                            dcc.Input(id='age-input', type='number', placeholder='Enter Age')]
                            ,className='three columns'),

                        html.Div(
                            [dbc.Label("Height"),
                            dcc.Input(id='height-input', type='number', placeholder='Enter Height')]

                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Weight"),
                            dcc.Input(id='weight-input', type='number', placeholder='Enter Weight')]

                            , className='three columns'),


                   ]

                   ,className='row', style=Div_style),
                 
                  ## secound Div with dropdown
                   html.Div(
                   [
                        html.Div(
                            [dbc.Label("Overweight Family History "),
                            dcc.Dropdown(
                                id='overweight-dropdown',
                                options=[

                                    {'label': 'yes', 'value': 'yes'},
                                    {'label': 'no', 'value': 'no'}

                                ],
                                value='no'
                            )]

                            , className='three columns'),


                        html.Div(
                             [dbc.Label("Consumption of Calorie"),
                             dcc.Dropdown(
                                id='calorie-dropdown',
                                options=[
                                    {'label': 'yes', 'value': 'yes'},
                                    {'label': 'no', 'value': 'no'}
                                ],
                                value='no'
                            )]
                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Vegetable Consumption"),
                            dcc.Slider(id='vegetable-slider', min=1, max=3, step=1, value=2.0)]

                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Number_Main_Meals"),
                            dcc.Slider(id='meals-slider', min=1, max=4, step=1, value=2)]

                            , className='three columns')
                   ]

                   ,className='row', style=Div_style),

                   ## third Div
                   html.Div(

                       [
                        html.Div(
                              [dbc.Label("Intermeal Consumption"),
                              dcc.Dropdown(
                                id='intermeal-dropdown',
                                options=[
                                    
                                    {'label': 'Always', 'value': 'Always'},
                                    {'label': 'Sometimes', 'value': 'Sometimes'},
                                    {'label': 'Frequently', 'value': 'Frequently'},
                                    {'label': 'no', 'value': 'no'}
                                ],
                                value='Always'
                            )]
                            , className='three columns'),


                        html.Div(
                             [dbc.Label("Smoke"),
                             dcc.Dropdown(
                                id='smoke-dropdown',
                                options=[
                                    {'label': 'yes', 'value': 'yes'},
                                    {'label': 'no', 'value': 'no'}
                                ],
                                value='no'
                            )]
                            , className='three columns'),

                        html.Div(

                            [dbc.Label("Consumption_Water_Daily"),
                            dcc.Slider(id='water-slider', min=1, max=3, step=1, value=2.0)]

                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Calorie Monitoring"),
                            dcc.Dropdown(
                                id='calorie-monitoring-dropdown',
                                options=[
                                    {'label': 'yes', 'value': 'yes'},
                                    {'label': 'no', 'value': 'no'}
                                ],
                                value='no'
                            )]

                            , className='three columns')
                   ]
                   ,className='row', style=Div_style),


                   ## four Div
                   html.Div(
                                       [
                        html.Div(
                            [dbc.Label("Physical_Activity_Frequency"),
                            dcc.Slider(id='activity-slider', min=0, max=3, step=1, value=2.0)]
                            , className='three columns'),


                        html.Div(
                            [dbc.Label("Time_Technology_Devices"),
                            dcc.Slider(id='technology-slider', min=0, max=2, step=1, value=2.0)]
                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Consumption_Alcohol"),
                            dcc.Dropdown(
                                id='alcohol-dropdown',
                                options=[
                                    
                                    {'label': 'Sometimes', 'value': 'Sometimes'},
                                    {'label': 'Frequently', 'value': 'Frequently'},
                                    {'label':  'Always' , 'value':  'Always'},
                                    {'label': 'no', 'value': 'no'}
                                ],
                                value="Sometimes"
                            )]

                            , className='three columns'),

                        html.Div(
                            [
                            dbc.Label("Transportation_Used"),
                            dcc.Dropdown(
                                id='transportation-dropdown',
                                options=[
                                    # ['Public_Transportation' 'Automobile' 'Walking' 'Motorbike' 'Bike']
                                    
                                    {'label': 'Public_Transportation', 'value': 'Public_Transportation'},
                                    {'label': 'Automobile', 'value': 'Automobile'},
                                    {'label': 'Walking', 'value': 'Walking'},
                                    {'label': 'Motorbike', 'value': 'Motorbike'},
                                    {'label': 'Bike', 'value': 'Bike'}
                                ],
                                value= 'Bike'
                            )
                            ]
                           , className='three columns')
                   ]
                   ,className='row', style=Div_style),
                  
                
                  ## Submit button
                   html.Div(
                        [
                        dbc.Button("Submit", id="submit-button", color="bluelight", className="mt-3", style={'margin': '20px'})

                        
                        ],
                        className='row'
                      ),
                  # # Output div for displaying predictions
                  html.Div(
                        id='prediction-output',
                        className='row', style={'padding': '20px', 'box-shadow': '0 4px 8px 0 rgba(0,0,0,0.2)'}
                        )
]  )






@app.callback(
    Output('prediction-output', 'children'),
    Input('submit-button', 'n_clicks'),
    State('gender-dropdown', 'value'),
    State('age-input', 'value'),
    State('height-input', 'value'),
    State('weight-input', 'value'),
    State('overweight-dropdown', 'value'),
    State('calorie-dropdown', 'value'),
    State('vegetable-slider', 'value'),
    State('meals-slider', 'value'),
    State('intermeal-dropdown', 'value'),
    State('smoke-dropdown', 'value'),
    State('water-slider', 'value'),
    State('calorie-monitoring-dropdown', 'value'),
    State('activity-slider', 'value'),
    State('technology-slider', 'value'),
    State('alcohol-dropdown', 'value'),
    State('transportation-dropdown', 'value')
)
def update_prediction(n_clicks, gender, age, height, weight, overweight_history, calorie_consumption, vegetable_consumption,
                      meals_per_day, intermeal_consumption, smoke, water_consumption, calorie_monitoring,
                      physical_activity, technology_usage, alcohol_consumption, transportation_used):
    if n_clicks is None:
        return None

    # Create DataFrame from input data
    data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Height': [height],
        'Weight': [weight],
        'family_history_with_overweight': [overweight_history],
        'FAVC': [calorie_consumption],
        'FCVC': [vegetable_consumption],
        'NCP': [meals_per_day],
        'CAEC': [intermeal_consumption],
        'SMOKE': [smoke],
        'CH2O': [water_consumption],
        'SCC': [calorie_monitoring],
        'FAF': [physical_activity],
        'TUE': [technology_usage],
        'CALC': [alcohol_consumption],
        'MTRANS': [transportation_used]
    })

    # Preprocess input data using the pipeline
   #     preprocessed_data = pipeline_loaded.transform(data)    --> error   

    # preprocessed_data = pipeline_loaded.named_steps['preprocessor'].transform(data)     
    # Make prediction using the model
    predicted_value = pipeline_loaded.predict(data)
    return f"Model Prediction: {predicted_value[0]}"




app.run_server(use_reloader=True)

In [6]:
import os

# File path
file_path = "D:/Projects ITI/DV/best_lgb_model.pkl"

# Check if file exists
if os.path.exists(file_path):
    print("File exists.")
else:
    print("File does not exist.")

# Get file size
file_size = os.path.getsize(file_path)
print(f"File size: {file_size} bytes")

# Try loading the file to verify its content
try:
    import joblib
    model = joblib.load(file_path)
    print("File loaded successfully.")
except Exception as e:
    print(f"Error loading file: {e}")


File exists.
File size: 12060508 bytes
Error loading file: 0


In [21]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier

# Load data
train_sample = pd.read_csv("train.csv")
train_orgin_extra = pd.read_csv("ObesityDataSet.csv")
train = pd.concat([train_sample, train_orgin_extra], ignore_index=True)
train.drop('id', axis=1, inplace=True)

# Remove duplicated data
train = train.drop_duplicates()

# Define categorical columns
categorical_columns = train.select_dtypes(include='object').columns
if 'NObeyesdad' in categorical_columns:
    categorical_columns = categorical_columns.drop(['NObeyesdad'])

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('ordinal', OrdinalEncoder(), categorical_columns),
        ('scaling', StandardScaler(), ['Age', 'Weight'])
    ], remainder='passthrough'
)

# Define pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LGBMClassifier(
        objective='multiclass',
        metric='multi_logloss',
        num_class=7,
        verbosity=-1,
        boosting_type='gbdt',
        random_state=42,
        learning_rate=0.03276219058578542,
        n_estimators=500,
        lambda_l1=0.009879324515507773,
        lambda_l2=0.04509276523818003,
        max_depth=10,
        colsample_bytree=0.451686663982718,
        subsample=0.9636469087931024,
        min_child_samples=28,
        n_jobs=-1
    ))
])

# Prepare data
X = train.drop(['NObeyesdad'], axis=1)
y = train['NObeyesdad']

# Train model
pipeline.fit(X, y)








############################################################################################



import dash
from dash import Input, Output, State, html, dcc, dash_table, MATCH, ALL, ctx
import dash_bootstrap_components as dbc
import dash_mantine_components as dmc
from dash_iconify import DashIconify
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, time, timedelta
import time as time_pck
import os
import dash_daq as daq
import joblib  # Import joblib to load the trained model

app = dash.Dash(external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])




# HOME_PATH = os.getcwd()
# DATA_PATH = os.path.join(HOME_PATH, 'data')
# MODELS_PATH = "D:/Projects ITI/DV/best_lgb_model.pkl"

# Now you can use this file_path variable wherever you need it
pipeline_loaded = joblib.load('model_pipeline.joblib')








# Define custom styles
custom_button_style = {
    'background-color': '#f0f0f0',
    'border-radius': '10px',
    'padding': '10px',
    'margin-bottom': '5px',
    'text-decoration': 'none',
    'display': 'inline-block',
}

Div_style = {
    'padding': '20px',
    'box-shadow': '0 4px 8px 0 rgba(0,0,0,0.2)',
    'marginTop': '10px',
    'backgroundColor': 'rgba(217, 218, 219, 0.6)',
    'color': 'black',
    'borderRadius': '25px',
    'display': 'flex',
    'justifyContent': 'space-between',  # Align divs horizontally
}

# App layout
app.layout = html.Div(children=[
                 ## first Div with drop down
                 html.Div(
                    [


                        html.Div(
                              [dbc.Label("Gender"),
                              dcc.Dropdown(
                                id='gender-dropdown',
                                options=[
                                    {'label': 'Female', 'value': 'Female'},
                                    {'label': 'Male', 'value': 'Male'}

                                ],
                                value="Female"
                            )]
                            , className='three columns'),


                        html.Div(
                            [dbc.Label("Age"),
                            dcc.Input(id='age-input', type='number', placeholder='Enter Age')]
                            ,className='three columns'),

                        html.Div(
                            [dbc.Label("Height"),
                            dcc.Input(id='height-input', type='number', placeholder='Enter Height')]

                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Weight"),
                            dcc.Input(id='weight-input', type='number', placeholder='Enter Weight')]

                            , className='three columns'),


                   ]

                   ,className='row', style=Div_style),
                 
                  ## secound Div with dropdown
                   html.Div(
                   [
                        html.Div(
                            [dbc.Label("Overweight Family History "),
                            dcc.Dropdown(
                                id='overweight-dropdown',
                                options=[
                                    {'label': 'Yes', 'value': 'Yes'},
                                    {'label': 'No', 'value': 'No'}

                                ],
                                value='Yes'
                            )]

                            , className='three columns'),


                        html.Div(
                             [dbc.Label("Consumption of Calorie"),
                             dcc.Dropdown(
                                id='calorie-dropdown',
                                options=[
                                    {'label': 'Yes', 'value': 'Yes'},
                                    {'label': 'No', 'value': 'No'}
                                ],
                                value="Yes"
                            )]
                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Vegetable Consumption"),
                            dcc.Slider(id='vegetable-slider', min=1, max=3, step=1, value=2.0)]

                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Number_Main_Meals"),
                            dcc.Slider(id='meals-slider', min=1, max=4, step=1, value=2)]

                            , className='three columns')
                   ]

                   ,className='row', style=Div_style),

                   ## third Div
                   html.Div(

                       [
                        html.Div(
                              [dbc.Label("Intermeal Consumption"),
                              dcc.Dropdown(
                                id='intermeal-dropdown',
                                options=[
                                    {'label': 'Always', 'value': 'Always'},
                                    {'label': 'Sometimes', 'value': 'Sometimes'},
                                    {'label': 'Frequently', 'value': 'Frequently'},
                                    {'label': 'No', 'value': 'No'}
                                ],
                                value="Always"
                            )]
                            , className='three columns'),


                        html.Div(
                             [dbc.Label("Smoke"),
                             dcc.Dropdown(
                                id='smoke-dropdown',
                                options=[
                                    {'label': 'Yes', 'value': 'Yes'},
                                    {'label': 'No', 'value': 'No'}
                                ],
                                value="Yes"
                            )]
                            , className='three columns'),

                        html.Div(

                            [dbc.Label("Consumption_Water_Daily"),
                            dcc.Slider(id='water-slider', min=1, max=3, step=1, value=2.0)]

                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Calorie Monitoring"),
                            dcc.Dropdown(
                                id='calorie-monitoring-dropdown',
                                options=[
                                    {'label': 'Yes', 'value': 'Yes'},
                                    {'label': 'No', 'value': 'No'}
                                ],
                                value="Yes"
                            )]

                            , className='three columns')
                   ]
                   ,className='row', style=Div_style),


                   ## four Div
                   html.Div(
                                       [
                        html.Div(
                            [dbc.Label("Physical_Activity_Frequency"),
                            dcc.Slider(id='activity-slider', min=0, max=3, step=1, value=2.0)]
                            , className='three columns'),


                        html.Div(
                            [dbc.Label("Time_Technology_Devices"),
                            dcc.Slider(id='technology-slider', min=0, max=2, step=1, value=2.0)]
                            , className='three columns'),

                        html.Div(
                            [dbc.Label("Consumption_Alcohol"),
                            dcc.Dropdown(
                                id='alcohol-dropdown',
                                options=[
                                    {'label': 'Sometimes', 'value': 'Sometimes'},
                                    {'label': 'Frequently', 'value': 'Frequently'},
                                    {'label': 'No', 'value': 'No'}
                                ],
                                value="Sometimes"
                            )]

                            , className='three columns'),

                        html.Div(
                            [
                            dbc.Label("Transportation_Used"),
                            dcc.Dropdown(
                                id='transportation-dropdown',
                                options=[
                                    {'label': 'Public_Transportation', 'value': 'Public_Transportation'},
                                    {'label': 'Automobile', 'value': 'Automobile'},
                                    {'label': 'Walking', 'value': ' Walking'},
                                    {'label': 'Motorbike', 'value': ' Motorbike'},
                                    {'label': 'Bike', 'value': ' Bike'}
                                ],
                                value= 'Walking'
                            )
                            ]
                           , className='three columns')
                   ]
                   ,className='row', style=Div_style),
                  
                
                  ## Submit button
                   html.Div(
                        [
                        dbc.Button("Submit", id="submit-button", color="bluelight", className="mt-3", style={'margin': '20px'})

                        
                        ],
                        className='row'
                      ),
                  # # Output div for displaying predictions
                  html.Div(
                        id='prediction-output',
                        className='row', style={'padding': '20px', 'box-shadow': '0 4px 8px 0 rgba(0,0,0,0.2)'}
                        )
]  )






@app.callback(
    Output('prediction-output', 'children'),
    Input('submit-button', 'n_clicks'),
    State('gender-dropdown', 'value'),
    State('age-input', 'value'),
    State('height-input', 'value'),
    State('weight-input', 'value'),
    State('overweight-dropdown', 'value'),
    State('calorie-dropdown', 'value'),
    State('vegetable-slider', 'value'),
    State('meals-slider', 'value'),
    State('intermeal-dropdown', 'value'),
    State('smoke-dropdown', 'value'),
    State('water-slider', 'value'),
    State('calorie-monitoring-dropdown', 'value'),
    State('activity-slider', 'value'),
    State('technology-slider', 'value'),
    State('alcohol-dropdown', 'value'),
    State('transportation-dropdown', 'value')
)
def update_prediction(n_clicks, gender, age, height, weight, overweight_history, calorie_consumption, vegetable_consumption,
                      meals_per_day, intermeal_consumption, smoke, water_consumption, calorie_monitoring,
                      physical_activity, technology_usage, alcohol_consumption, transportation_used):
    if n_clicks is None:
        return None

    # Create DataFrame from input data
    data = pd.DataFrame({
        'Gender': [gender],
        'Age': [age],
        'Height': [height],
        'Weight': [weight],
        'family_history_with_overweight': [overweight_history],
        'FAVC': [calorie_consumption],
        'FCVC': [vegetable_consumption],
        'NCP': [meals_per_day],
        'CAEC': [intermeal_consumption],
        'SMOKE': [smoke],
        'CH2O': [water_consumption],
        'SCC': [calorie_monitoring],
        'FAF': [physical_activity],
        'TUE': [technology_usage],
        'CALC': [alcohol_consumption],
        'MTRANS': [transportation_used]
    })

    
    categorical_columns = data.select_dtypes(include='object').columns
    data[categorical_columns] = ordinal_encoder.fit_transform(data[categorical_columns])
    data[categorical_columns] = data[categorical_columns].astype(int)
    data['NCP'] = data['NCP'].astype(int)
    scaler1=StandardScaler()
    age_data = np.array(data['Age']).reshape(-1, 1)
    data['Age'] = scaler1.fit_transform(age_data)
    scaler2=StandardScaler()
    weight_data = np.array(data['Weight']).reshape(-1, 1)
    data['Weight']=scaler2.fit_transform(weight_data)
    
    # Preprocess input data using the pipeline
    # preprocessed_data =  pipeline_loaded.transform(data)
     
    # Make prediction using the model
    predicted_value = pipeline_loaded.predict(data)

    return f"Model Prediction: {predicted_value}"




app.run_server(use_reloader=True)

In [7]:
import os
import pickle

# Step 1: Check if the file exists
file_path = "D:/Projects ITI/DV/best_lgb_model.pkl"
if not os.path.exists(file_path):
    print("Error: File not found.")
else:
    print("File exists.")

    # Step 2: Check the file size
    file_size = os.path.getsize(file_path)
    print(f"File size: {file_size} bytes")

    # Step 3: Try loading the file with joblib
    try:
        model = joblib.load(file_path)
        print("Model loaded successfully with joblib.")
    except Exception as e:
        print(f"Error loading file with joblib: {e}")

    # Step 4: Try loading the file with pickle
    try:
        with open(file_path, 'rb') as f:
            model = pickle.load(f)
            print("Model loaded successfully with pickle.")
    except Exception as e:
        print(f"Error loading file with pickle: {e}")

    # Step 5: Inspect file content manually (optional)
    # You can open the file in a text editor or a hex editor to inspect its content.

    # Step 6: Recreate the model (if necessary)
    # If none of the loading methods work and the file seems corrupted, consider recreating the model.


File exists.
File size: 12060508 bytes
Error loading file with joblib: 0
Error loading file with pickle: invalid load key, '\x02'.


In [12]:


# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

In [11]:
y_pred

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype=object)